### Datenanreicherung – Artikeltexte scrapen

Dieses Notebook ruft Artikeltexte über die zuvor extrahierten Links ab. Verarbeitet werden dabei nur Links aus HTML-Dateien ausgewählter Medien, die eines der definierten Keywords BVG, HVV (inkl. VHH, Hochbahn) oder MVG enthalten.

Die wichtigsten Schritte:
- Bereinigung der Linkliste 
- Web Scraping der Artikeltexte über die gespeicherten URLs
- Speicherung der Artikeltexte als CSV-Datei für die Assoziationsanalyse
- Logging erfolgreicher und fehlerhafter Abrufe zur Nachvollziehbarkeit 
- Bereinigung und Zerlegung der Artikeltexte in Wörter für die Wortfrequenzanalyse

Dieses Skript setzt voraus, dass die HTML-Linkdaten bereits extrahiert und gefiltert wurden (siehe: 04_artikel_links_suchen.ipynb)

#### 1. Import benötigte Pakete

In [72]:
# Standard
import os # Dateipfaden
import pandas as pd # Tabellenverarbeitung (DataFrames)

# HTML-Verarbeitung & Scraping
from bs4 import BeautifulSoup # HTML analysieren
import requests # HTTP-Anfragen
import time # Pausen zwischen HTTP-Anfragen

# Eigene Funktionen (ausgelagert)
import sys  # Systemfunktionen 
sys.path.append("..") # Pfad zu .py Datei
from scripts.datenaufbereitung import load_stopwords
from scripts.cluster_mapping import medium_to_cluster
from scripts.textbereinigung import clean_and_split_text

In [73]:
# Pfade
# Projektverzeichnis
PROJECT_ROOT = r"D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping"

# Datei für die Analyse
DATAPATH = os.path.join(PROJECT_ROOT, "output", "artikel_links_headlines.csv")

# Output-Pfade
OUTPUT_PATH = os.path.join(PROJECT_ROOT, "output") 

# Bereinigte DATAPATH
CSV_PATH = os.path.join(OUTPUT_PATH, "artikel_links_headlines_clean.csv")
# Rohtexte nach Scraping
CSV_PATH_RAW_ARTIKEL = os.path.join(OUTPUT_PATH, "artikeltexte_raw.csv")
FAILED_PATH_RAW_ARTIKEL = os.path.join(OUTPUT_PATH, "artikeltexte_failed.csv")

# Zerlegte Texte nach Scraping
CSV_PATH_COUNTS = os.path.join(OUTPUT_PATH, "artikel_wortfrequenz.csv")

#### 2. Datenexploration

In [74]:
# CSV-Dateien einlesen
df_artikel = pd.read_csv(DATAPATH)

In [75]:
# Anzahl Zeilen und Spalten
df_artikel.shape

(5593, 6)

In [76]:
# Spaltennamen
df_artikel.columns

Index(['filename', 'source', 'date', 'url', 'headline', 'cluster'], dtype='object')

In [77]:
# Prüfung fehlende Werte
df_artikel.isnull().sum()

filename    0
source      0
date        0
url         0
headline    0
cluster     0
dtype: int64

In [78]:
# Erste 10 Artikelzeilen anzeigen
df_artikel.head(10)

,filename,source,date,url,headline,cluster
0,2021-04-01-sz.html,sz,2021-04-01,https://www.sueddeutsche.de/muenchen/muenchen-...,Wartungsarbeiten der MVGEinschränkungen im Apr...,Große Medien
1,2021-04-01-tagesspiegel.html,tagesspiegel,2021-04-01,https://checkpoint.tagesspiegel.de/langmeldung...,Zweifel an britischem Impfstoff:Verwaltung drä...,Regional
2,2021-04-01-tagesspiegel.html,tagesspiegel,2021-04-01,https://www.tagesspiegel.de/themen/bvg/,BVG,Regional
3,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/berlin/streit-um-s...,ExklusivStreit um Schutz vor Hackerangriffen:B...,Regional
4,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/berlin/streit-um-s...,mehr,Regional
5,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/berlin/pruefung-vo...,Prüfung von Cyberabwehr:BVG eskaliert im Konfl...,Regional
6,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/berlin/opposition-...,Opposition attackiert Berliner Wirtschaftssena...,Regional
7,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/gesellschaft/queer...,BVG-Chefin Kreienkamp über ihr Coming-out:„Vie...,Regional
8,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/berlin/streit-um-s...,09:51Streit um Schutz vor HackerangriffenBVG z...,Regional
9,2021-04-02-tagesspiegel.html,tagesspiegel,2021-04-02,https://www.tagesspiegel.de/themen/bvg/,BVG,Regional


In [79]:
# Kurze Headlines filtern und anzeigen
kurze_headlines = df_artikel[df_artikel["headline"].str.len() <= 3]
print(kurze_headlines[["headline", "url"]])

     headline                                                url
2         BVG            https://www.tagesspiegel.de/themen/bvg/
9         BVG            https://www.tagesspiegel.de/themen/bvg/
16        BVG            https://www.tagesspiegel.de/themen/bvg/
17        BVG            https://www.tagesspiegel.de/themen/bvg/
23        BVG            https://www.tagesspiegel.de/themen/bvg/
...       ...                                                ...
1800      BVG            https://www.tagesspiegel.de/themen/bvg/
1804      BVG            https://www.tagesspiegel.de/themen/bvg/
4913      BVG         https://www.berliner-zeitung.de/topics/bvg
4915      BVG         https://www.berliner-zeitung.de/topics/bvg
5504       52  https://www.zeit.de/arbeit/2025-04/verdi-bvg-b...

[527 rows x 2 columns]


In [80]:
# Anzahl kurze Headlines nach Medium
kurze_headlines["source"].value_counts()

source
tagesspiegel    524
berliner          2
zeit              1
Name: count, dtype: int64

In [81]:
# 5 Links mit häufigster Nennung beim Tagesspiegel anzeigen
df_artikel[df_artikel["source"] == "tagesspiegel"]["url"].value_counts().head(5)

url
https://www.tagesspiegel.de/themen/bvg/                                                                                                                                   528
https://www.tagesspiegel.de/berlin/falsch-gehende-uhren-in-berlin-als-die-bvg-nicht-richtig-tickte/28249414.html                                                           15
https://www.tagesspiegel.de/berlin/plotzlich-war-ich-die-stimme-der-bvg-synchronsprecherin-philippa-jarke-macht-die-ansagen-in-berlins-bussen-und-bahnen-12977182.html     12
https://www.tagesspiegel.de/berlin/kuerzungen-bei-der-bvg-31-berliner-buslinien-fahren-ab-montag-seltener-wegen-corona-und-personalmangel/28600266.html                    11
https://www.tagesspiegel.de/berlin/polizei-justiz/19-jaehriger-erleidet-kopfverletzungen-radfahrer-in-berlin-bei-unfall-mit-bvg-bus-schwer-verletzt/27277756.html          11
Name: count, dtype: int64

In [82]:
# Kurze Headlines entfernen
df_artikel = df_artikel[df_artikel["headline"].str.len() > 3]

In [83]:
# Ergebnis anzeigen
print("[INFO] Anzahl verbleibender Links:", len(df_artikel))

[INFO] Anzahl verbleibender Links: 5066


In [84]:
# Duplikate prüfen
df_artikel["url"].duplicated().sum()

np.int64(3195)

In [85]:
# Duplikate anzeigen 
df_artikel[df_artikel.duplicated(subset="url", keep=False)].sort_values("url")

,filename,source,date,url,headline,cluster
3081,2023-11-23-taz.html,taz,2023-11-23,https://blogs.taz.de/zylinderkopf/bvg-urteil-z...,Zylinderkopf-Dichtung: Zur Schuldenbremse„Denn...,Große Medien
3082,2023-11-24-taz.html,taz,2023-11-24,https://blogs.taz.de/zylinderkopf/bvg-urteil-z...,Zylinderkopf-Dichtung: Zur Schuldenbremse„Denn...,Große Medien
1092,2022-04-12-tagesspiegel.html,tagesspiegel,2022-04-12,https://checkpoint.tagesspiegel.de/encore/2jf2...,Schwan-Alarm am Lietzensee:Berlin errichtet we...,Regional
1093,2022-04-12-tagesspiegel.html,tagesspiegel,2022-04-12,https://checkpoint.tagesspiegel.de/encore/2jf2...,mehr,Regional
1363,2022-06-02-tagesspiegel.html,tagesspiegel,2022-06-02,https://checkpoint.tagesspiegel.de/langmeldung...,Regulär und weniger regulär:BVG und Bahn kontr...,Regional
...,...,...,...,...,...,...
2109,2022-12-13-zeit.html,zeit,2022-12-13,https://www.zeit.de/mobilitaet/2022-12/sunglid...,124 Kommentare,Große Medien
1504,2022-07-04-handelsblatt.html,handelsblatt,2022-07-04,https://www.zeit.de/news/2022-07/04/gericht-bv...,Berliner Verkehrsbetriebe:Gericht,Wirtschaft
1498,2022-07-04-zeit.html,zeit,2022-07-04,https://www.zeit.de/news/2022-07/04/gericht-bv...,09:51Berliner Verkehrsbetriebe:Gericht: BVG da...,Große Medien
2633,2023-05-03-zeit.html,zeit,2023-05-03,https://www.zeit.de/wirtschaft/2023-05/mobilit...,"Podcast: Mobilitätswende: ""Sie wollten einfach...",Große Medien


In [86]:
# Duplikate entfernen
df_artikel = df_artikel.drop_duplicates(subset="url")

In [87]:
# Ergebnis anzeigen
print("[INFO] Anzahl verbleibender Links:", len(df_artikel))

[INFO] Anzahl verbleibender Links: 1871


In [88]:
# Ergebnis als CSV-Datei exportieren
df_artikel.to_csv(CSV_PATH, index=False)
print(f"[INFO] Links gespeichert unter: {CSV_PATH}")

[INFO] Links gespeichert unter: D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping\output\artikel_links_headlines_clean.csv


#### 3. Funktionen für Web Scraping definieren

In [89]:
# Funktion: "BVG", "HVV", "VHH", "Hochbahn" oder "MVG" zurückgeben, wenn das entsprechende Keyword im Text enthalten ist
def detect_keyword(text):
    text = text.lower()
    if "bvg" in text:
        return "bvg"
    elif "mvg" in text:
        return "mvg"
    elif "hvv" in text:
        return "hvv"
    elif "vhh" in text:
        return "vhh"
    elif "hochbahn" in text:
        return "hochbahn"
    return "None"

In [90]:
# Funktion: Artikeltext von einer URL laden und Ergebnis-Dictionary zurückgeben
def scrape_article(entry):
    # URL zurückgeben oder "" wenn nicht vorhanden
    url = entry.get("url", "")

    if not url:
        # Kein Abruf möglich
        return None  

    try:
        # HTTP-Anfrage mit User-Agent senden
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

        # Wenn Abruf erfolgreich (HTTP-Status 200)
        if response.status_code == 200:
            # HTML parsen
            soup = BeautifulSoup(response.text, "html.parser")
            # Textabsätze finden
            paragraphs = soup.find_all("p")
            # Text zusammensetzen
            text = " ".join(p.get_text(strip=True) for p in paragraphs)

            # Rückgabe als Dictionary 
            if text.strip():
                return {
                    "filename": entry.get("filename", ""),
                    "url": url,
                    "source": entry.get("source", ""),
                    "date": entry.get("date", ""),  
                    "headline": entry.get("headline", ""),
                    "keyword": detect_keyword(entry.get("headline", "")),
                    "text": text,
                    "cluster": medium_to_cluster.get(entry.get("source", ""), "Unbekannt")
                }

        # Wenn zu viele Anfragen (HTTP-Status 429): warten und erneut versuchen
        elif response.status_code == 429:
            time.sleep(10)
            return scrape_article(entry)

    except Exception:
        pass  

    return None  

#### 4. Scraping der Artikeltexte

In [91]:
# Ergebnis-Listen vorbereiten
successful_articles = []
failed_articles = []
article_urls = df_artikel.to_dict(orient="records")

# Artikeltexte iterativ abrufen
for entry in article_urls:
    result = scrape_article(entry)

    if result:
        successful_articles.append(result)
    else:
        failed_articles.append(entry)

    # Kurze Pause zwischen Anfragen
    time.sleep(1.5)  

# Ergebnis anzeigen
print(f"[INFO] Erfolgreich extrahierte Artikel: {len(successful_articles)}")
print(f"[INFO] Fehlgeschlagene Versuche: {len(failed_articles)}")

[INFO] Erfolgreich extrahierte Artikel: 1843
[INFO] Fehlgeschlagene Versuche: 28


#### 5. Speicherung der Artikeltexte

In [92]:
# Erfolgreiche Artikel als DataFrame speichern
if successful_articles:
    df_scraped = pd.DataFrame(successful_articles)
    df_scraped.to_csv(CSV_PATH_RAW_ARTIKEL, index=False)
    print(f"[INFO] Artikeltexte gespeichert unter: {CSV_PATH_RAW_ARTIKEL}")
else:
    print("[WARNING] Keine Artikeltexte erfolgreich geladen")

[INFO] Artikeltexte gespeichert unter: D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping\output\artikeltexte_raw.csv


In [93]:
# Fehlgeschlagene Artikel speichern
pd.DataFrame(failed_articles).to_csv(FAILED_PATH_RAW_ARTIKEL, index=False)
print(f"[INFO] Fehlgeschlagene Artikel gespeichert unter: {FAILED_PATH_RAW_ARTIKEL}")

[INFO] Fehlgeschlagene Artikel gespeichert unter: D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping\output\artikeltexte_failed.csv


#### 6. Bereinigung und Zerlegung der Artikeltexte in Wörter

In [94]:
# Vorbereitung
# Stoppwörter laden
stopwords_list = load_stopwords()

# Funktion: Einen Artikel verarbeiten und Wortfrequenzen zählen
def process_article(row):
    try:
        text = row["text"]
        words = clean_and_split_text(text, stopwords_list)

        # Wortfrequenzen
        count = pd.Series(words).value_counts()
        count_df = count.to_frame()
        count_df.columns = ["count"]
        count_df["word"] = count_df.index
        count_df["source"] = row["source"]
        count_df["cluster"] = medium_to_cluster.get(row["source"], "Unbekannt")
        count_df["date"] = row["date"]
        count_df["url"] = row["url"]  
        count_df["year"] = pd.to_datetime(row["date"]).year

        return count_df
    except Exception as e:
        print(f"[WARNING] Fehler bei Artikel {row.get('url', 'unbekannt')}: {e}")
        return pd.DataFrame()

In [95]:
# Verarbeitungsfunktion anwenden
# Artikel laden
df_scraped = pd.read_csv(CSV_PATH_RAW_ARTIKEL)

# Wortfrequenzen sammeln
collection = []
for _, row in df_scraped.iterrows():
    count_df = process_article(row)
    if not count_df.empty:
        collection.append(count_df)

# Alle Ergebnisse zusammenführen
if collection:
    df_counts = pd.concat(collection, ignore_index=True)
    df_counts.to_csv(CSV_PATH_COUNTS, index=False)
    print(f"[INFO] Wortfrequenzen gespeichert unter: {CSV_PATH_COUNTS}")
    print("Anzahl verarbeiteter Artikel:", df_scraped.shape[0])
    
else:
    print("[WARNING] Keine Wortfrequenzen berechnet.")

[INFO] Wortfrequenzen gespeichert unter: D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping\output\artikel_wortfrequenz.csv
Anzahl verarbeiteter Artikel: 1843


In [96]:
# Ergebnis anzeigen
print(f"[INFO] Anzahl erfolgreich verarbeiteter Artikel: {len(collection)}")

[INFO] Anzahl erfolgreich verarbeiteter Artikel: 1843
